In [17]:
# 卷积神经网络的设计是用于探索图像数据，本节我们将以图像为例。
import torch
from torch import nn
from d2l import torch as d2l

## 1.卷积操作
输入源图像 X 和卷积核 K，输出张量 Y

In [18]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()

    return Y

## 2. 卷积层
卷积层对*输入*和*卷积核权重*进行互相关运算，并在添加*标量偏置*之后产生输出。
两个主要参数是 `卷积核权重` 和 `标量偏置` 。
同样，需要在训练模型前对其**随机初始化**。

In [19]:
# 卷积层在代码中表现为一个 nn.Module 的子类
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super.__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias


## 3. 边缘检测
简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘。
构造一个 $6*8$ 像素的黑白图像。中间四列为黑色(0)，其余像素为白色(1)。

In [20]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

构造一个高度为 1 、宽度为 2 的卷积核 K。
当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [21]:
K = torch.tensor([[1.0, -1.0]])
K

tensor([[ 1., -1.]])

开始卷积

In [22]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

由输出可知（列[1] 为 1;列[-2]为 -1），该卷积核能检测出**竖向**的边缘

尝试将 X 转置，并再次卷积：

In [23]:
# Y = corr2d(X.T, K)
# Y

转置消失，说明横向边缘无法检测

## 4. 卷积核
矛盾在于，当任务要求变得复杂，不可能再*手动*设置卷积核权重和偏置。
现在需要解决的问题是：已知 X 和 Y, 能否生成 K 和 bias ?

先构造一个卷积层，并将其卷积核初始化为随机张量。
接下来，在每次迭代中，我们比较 Y 与卷积层输出的平方误差，
然后计算梯度来更新卷积核。

In [24]:
# 简单起见，暂不考虑 bias
# 1 个输入通道，1 个输出通道，形状为（1，2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为 1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    loss = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    loss.sum().backward()

    # 迭代卷积核
    conv2d.weight.data[:] -=lr*conv2d.weight.grad

    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {loss.sum():.3f}')

epoch 2, loss 23.355
epoch 4, loss 8.351
epoch 6, loss 3.217
epoch 8, loss 1.283
epoch 10, loss 0.520


In [26]:
# 查看此时的卷积核权重
conv2d.weight.data.reshape(1,2)

tensor([[ 0.9142, -1.0625]])

## 5. 互相关和卷积

## 6. 特征映射和感受野

感受野是 **输出 Tensor** 的每个元素的属性。
一个输出层元素的感受野是 **输入 Tensor** 上被当前位置的卷积核 **覆盖部分** 元素。

## 7.练习

In [27]:
M =  torch.eye(8)
M

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1.]])